---
## 1. Architecture Overview

The voice chat application uses a **three-tier architecture**:

```
┌─────────────────┐     WebSocket     ┌──────────────────┐     WebSocket     ┌─────────────────────┐
│   Browser UI    │ ◄──────────────► │  Backend Server  │ ◄──────────────► │ Azure OpenAI        │
│  (JavaScript)   │                  │    (Python)      │                  │ Realtime API        │
└─────────────────┘                  └──────────────────┘                  └─────────────────────┘
        │                                    │                                      │
        │  • Captures microphone             │  • Proxies messages                  │  • Speech-to-Text
        │  • Plays audio response            │  • Hides API keys                    │  • LLM Processing
        │  • Manages UI state                │  • Rate limiting                     │  • Text-to-Speech
        │                                    │  • Session management                │
```

### Why a Backend Proxy?

1. **Security**: API keys never leave the server
2. **Rate Limiting**: Control usage per user
3. **Logging**: Monitor and debug conversations
4. **Flexibility**: Add custom logic, filters, or transformations

---
## 2. WebSocket Fundamentals

### What is a WebSocket?

WebSocket is a **bidirectional, full-duplex communication protocol** over a single TCP connection. Unlike HTTP (request-response), WebSockets allow both client and server to send messages at any time.

### HTTP vs WebSocket

| Aspect | HTTP | WebSocket |
|--------|------|----------|
| Communication | Request-Response | Bidirectional |
| Connection | New connection per request | Persistent connection |
| Overhead | Headers on every request | Minimal framing |
| Use Case | REST APIs, web pages | Real-time apps, streaming |
| Latency | Higher | Lower |

### Why WebSocket for Voice Chat?

Voice chat requires **real-time, continuous data streaming** in both directions:
- 🎤 User's voice → Server → Azure (continuous audio stream)
- 🔊 Azure's response → Server → User (continuous audio stream)

HTTP would require polling or multiple requests, adding unacceptable latency.

In [ ]:
# WebSocket Connection Lifecycle
# This is conceptual code - won't run without a WebSocket server

import asyncio

# Simulating WebSocket connection lifecycle
class WebSocketLifecycle:
    """
    Demonstrates the WebSocket connection lifecycle.
    
    1. CONNECTING: Initial handshake
    2. OPEN: Connection established, can send/receive
    3. CLOSING: Graceful shutdown initiated
    4. CLOSED: Connection terminated
    """
    
    states = ['CONNECTING', 'OPEN', 'CLOSING', 'CLOSED']
    
    def __init__(self):
        self.state = 'CONNECTING'
        
    def transition(self, new_state):
        print(f"State: {self.state} → {new_state}")
        self.state = new_state

# Demonstrate state transitions
ws = WebSocketLifecycle()
print("WebSocket Connection Lifecycle:")
print("="*40)
ws.transition('OPEN')      # After successful handshake
ws.transition('CLOSING')   # Close initiated
ws.transition('CLOSED')    # Connection terminated

---
## 3. Azure OpenAI Realtime API

### What is the Realtime API?

Azure OpenAI's **Realtime API** provides:
- 🎤 **Speech-to-Text**: Transcribes audio in real-time
- 🧠 **LLM Processing**: Generates intelligent responses
- 🔊 **Text-to-Speech**: Converts response to natural speech

All in a **single WebSocket connection** with sub-second latency!

### API Endpoint Structure

```
wss://{endpoint}/openai/realtime
    ?api-version={version}
    &deployment={deployment-name}
    &api-key={your-api-key}
```

### Supported Models

- `gpt-4o-realtime-preview` - Best quality, lower latency
- Models are deployed to your Azure OpenAI resource

In [ ]:
# Building the Azure Realtime API URL
# This shows how to construct the WebSocket URL for Azure OpenAI

def build_azure_realtime_url(
    endpoint: str,
    deployment: str,
    api_key: str,
    api_version: str = "2024-10-01-preview"
) -> str:
    """
    Build the WebSocket URL for Azure OpenAI Realtime API.
    
    Args:
        endpoint: Your Azure OpenAI endpoint (https://...)
        deployment: Name of your gpt-4o-realtime deployment
        api_key: Your Azure OpenAI API key
        api_version: API version to use
    
    Returns:
        WebSocket URL for connection
    """
    # Convert HTTPS to WSS (secure WebSocket)
    ws_endpoint = endpoint.replace('https://', 'wss://').rstrip('/')
    
    url = (
        f"{ws_endpoint}/openai/realtime"
        f"?api-version={api_version}"
        f"&deployment={deployment}"
        f"&api-key={api_key}"
    )
    return url

# Example (with fake credentials)
example_url = build_azure_realtime_url(
    endpoint="https://my-openai.openai.azure.com",
    deployment="gpt-4o-realtime",
    api_key="abc123..."
)

print("Azure Realtime API URL Structure:")
print("="*50)
# Show URL without exposing the key
safe_url = example_url.split('&api-key=')[0] + '&api-key=***'
print(safe_url)

---
## 4. Audio Encoding & Processing

### Audio Format Requirements

Azure OpenAI Realtime API expects audio in specific formats:

| Property | Value |
|----------|-------|
| Format | PCM (Pulse Code Modulation) |
| Sample Rate | 24000 Hz (24 kHz) |
| Bit Depth | 16-bit |
| Channels | Mono (1 channel) |
| Encoding | Base64 (for JSON messages) |

### Audio Pipeline

```
Browser Microphone          Server              Azure
      │                       │                   │
      │  AudioWorklet         │                   │
      │  captures PCM         │                   │
      ▼                       │                   │
  Float32 samples             │                   │
      │                       │                   │
      │  Downsample to        │                   │
      │  24kHz if needed      │                   │
      ▼                       │                   │
  Int16 PCM                   │                   │
      │                       │                   │
      │  Base64 encode        │                   │
      ▼                       │                   │
  JSON message ───────────────►───────────────────►
                              │                   │
                              │                   │ Process
                              │                   ▼
  ◄────────────────────────────◄──────────────────
      │                       │  Response audio
      │  Decode & play        │
      ▼                       │
  Speaker output              │
```

---
## 4a. Microsoft Agent Framework (Text Mode)

### What is Microsoft Agent Framework?

The **Microsoft Agent Framework** provides a unified way to build AI agents across Python and .NET:

- 🤖 **ChatAgent**: High-level abstraction for chat-based AI interactions
- 🧵 **AgentSession**: Manages conversation history across multiple turns
- 🔧 **Tool Support**: Native functions, OpenAPI, and MCP (Model Context Protocol)
- ☁️ **Multi-Provider**: Azure OpenAI, OpenAI, Microsoft Foundry, and more

### Why Use Agent Framework?

| Feature | Direct API Calls | Agent Framework |
|---------|-----------------|-----------------|
| Conversation Memory | Manual management | Built-in sessions |
| Tool/Function Calling | Complex setup | Declarative |
| Multi-turn Context | Implement yourself | Automatic |
| Streaming Responses | Manual parsing | Built-in support |
| Cross-platform | Separate implementations | Same patterns (Python & .NET) |

### Package Installation (Python)

```bash
# The --pre flag is required while Agent Framework is in preview
pip install agent-framework-core agent-framework-azure-ai --pre
```

### Key Classes

| Class | Purpose |
|-------|---------|
| `AzureOpenAIChatClient` | Connect to Azure OpenAI with explicit settings |
| `create_agent()` | Create an agent from the chat client |
| `agent.create_session()` | Create conversation session for multi-turn chat |
| `agent.run()` | Run agent and get response |
| `agent.run_stream()` | Run agent with streaming response |

### Comparison with .NET

| Python | .NET |
|--------|------|
| `AzureOpenAIChatClient` | `AzureOpenAIClient` |
| `client.create_agent()` | `chatClient.CreateAIAgent()` |
| `agent.run()` | `agent.RunAsync()` |
| `agent.run_stream()` | `agent.RunStreamingAsync()` |
| `agent.create_session()` | `agent.CreateSessionAsync()` |

In [ ]:
# Microsoft Agent Framework - AzureOpenAIChatClient Pattern (as used in server.py)
# This demonstrates the pattern used in the Voice Chat backend for text mode

AGENT_SERVICE_CODE = '''
from agent_framework.azure import AzureOpenAIChatClient

# Global agent instance (singleton pattern)
_chat_agent = None
_chat_client = None

async def get_chat_agent():
    """
    Get or create the ChatAgent instance (singleton pattern like .NET).
    Uses Microsoft Agent Framework with Azure OpenAI.
    
    Pattern follows:
    https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai
    """
    global _chat_agent, _chat_client
    
    if _chat_agent is not None:
        return _chat_agent
    
    # Create the Azure OpenAI Chat Client with explicit settings
    # This mirrors the .NET pattern: AzureOpenAIClient -> GetChatClient -> CreateAIAgent
    _chat_client = AzureOpenAIChatClient(
        endpoint=AZURE_ENDPOINT,
        deployment_name=AZURE_CHAT_DEPLOYMENT,
        api_key=AZURE_API_KEY,
    )
    
    # Create agent from the chat client (like .NET's CreateAIAgent)
    _chat_agent = _chat_client.create_agent(
        instructions="You are a helpful assistant. Respond naturally and concisely.",
    )
    
    return _chat_agent


async def handle_text_mode(websocket, session_id: str):
    """Handle Text Mode connection using Microsoft Agent Framework."""
    
    agent = await get_chat_agent()
    
    # Create a new session for this conversation session
    # (like .NET's agent.CreateSessionAsync())
    session = await agent.create_session()
    
    async for message in websocket:
        data = json.loads(message)
        user_message = data.get('content', '')
        
        # Run the agent with the user's message (mirrors .NET's agent.RunAsync)
        result = await agent.run(user_message, session=session)
        
        # AgentRunResponse can be converted to string for the text content
        response_text = str(result) if result else "No response generated."
        
        await websocket.send(json.dumps({
            'type': 'text_response',
            'content': response_text
        }))
'''

print("AzureOpenAIChatClient Pattern - Text Mode Handler:")
print("="*60)
print(AGENT_SERVICE_CODE)

In [ ]:
# Agent session for Multi-turn Conversations
# AgentSession maintains conversation context across multiple user interactions

THREAD_PATTERN_CODE = '''
# Multi-turn conversation using AgentSession
# Pattern from: https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai/azure_chat_client_with_session.py

from agent_framework.azure import AzureOpenAIChatClient

async def demo_multi_turn_conversation():
    """Demonstrate conversation memory with AgentSession."""
    
    # Create client and agent
    client = AzureOpenAIChatClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
    )
    
    agent = client.create_agent(
        instructions="You are a helpful assistant.",
    )
    
    # Create a new session for conversation history
    session = await agent.create_session()
    
    # First turn
    result1 = await agent.run("My name is Alice", session=session)
    print(f"Agent: {result1}")
    # Agent: "Nice to meet you, Alice!"
    
    # Second turn - agent remembers the context via thread
    result2 = await agent.run("What is my name?", session=session)
    print(f"Agent: {result2}")
    # Agent: "Your name is Alice!"
    
    # Without thread, each call would be independent
    # The thread automatically manages the conversation history


# Adding tools/functions to the agent
from typing import Annotated
from random import randint
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

# Agent with tools
agent_with_tools = client.create_agent(
    instructions="You are a helpful weather assistant.",
    tools=[get_weather],  # Add callable functions as tools
)

# The agent can now call get_weather when user asks about weather
result = await agent_with_tools.run("What's the weather like in Seattle?")
# Agent will call get_weather("Seattle") and incorporate the result
'''

print("AgentSession - Multi-turn Conversations & Tools:")
print("="*60)
print(THREAD_PATTERN_CODE)

In [ ]:
import base64
import struct
import math

# Audio processing concepts

def float32_to_int16(samples: list) -> bytes:
    """
    Convert float32 audio samples (-1.0 to 1.0) to int16 PCM.
    This is what happens in the browser's AudioWorklet.
    
    Args:
        samples: List of float32 samples in range [-1.0, 1.0]
    
    Returns:
        Bytes containing int16 PCM data
    """
    int16_samples = []
    for sample in samples:
        # Clamp to valid range
        clamped = max(-1.0, min(1.0, sample))
        # Scale to int16 range (-32768 to 32767)
        int16_value = int(clamped * 32767)
        int16_samples.append(int16_value)
    
    # Pack as little-endian int16
    return struct.pack(f'<{len(int16_samples)}h', *int16_samples)


def pcm_to_base64(pcm_bytes: bytes) -> str:
    """
    Convert PCM bytes to base64 string for JSON transport.
    
    Args:
        pcm_bytes: Raw PCM audio bytes
    
    Returns:
        Base64 encoded string
    """
    return base64.b64encode(pcm_bytes).decode('utf-8')


# Demonstrate with a simple sine wave (440 Hz = A4 note)
sample_rate = 24000  # 24 kHz as required by Azure
frequency = 440  # Hz
duration = 0.01  # 10 milliseconds

# Generate sine wave samples
num_samples = int(sample_rate * duration)
sine_wave = [math.sin(2 * math.pi * frequency * i / sample_rate) for i in range(num_samples)]

# Convert to int16 PCM
pcm_data = float32_to_int16(sine_wave)

# Convert to base64 for JSON transport
base64_audio = pcm_to_base64(pcm_data)

print("Audio Encoding Example (440 Hz sine wave, 10ms):")
print("="*50)
print(f"Sample rate: {sample_rate} Hz")
print(f"Number of samples: {num_samples}")
print(f"PCM bytes: {len(pcm_data)} bytes")
print(f"Base64 length: {len(base64_audio)} characters")
print(f"\nBase64 preview: {base64_audio[:50]}...")

---
## 5. Session Management

### Why Session Management?

Real-time voice applications need to:
- Track active connections
- Associate users with their sessions
- Clean up resources when connections close
- Implement rate limiting per user

### Session Structure

```python
session = {
    'session_id': 'uuid-string',
    'user_id': 'user-identifier',
    'mode': 'voice' | 'text',
    'created_at': datetime,
    'azure_ws': WebSocket,  # Connection to Azure
    'client_ws': WebSocket, # Connection to browser
    'message_count': 0
}
```

In [ ]:
import uuid
from datetime import datetime
from typing import Dict, Set, Optional
from collections import defaultdict

class SessionManager:
    """
    Manages voice chat sessions with rate limiting.
    
    Features:
    - Create/cleanup sessions
    - Track sessions per user
    - Enforce connection limits
    - Rate limiting
    """
    
    MAX_CONNECTIONS_PER_USER = 3
    MAX_REQUESTS_PER_MINUTE = 60
    
    def __init__(self):
        self.sessions: Dict[str, dict] = {}
        self.user_connections: Dict[str, Set[str]] = defaultdict(set)
        self.user_requests: Dict[str, list] = defaultdict(list)
    
    def create_session(self, user_id: str, mode: str) -> str:
        """Create a new session for a user."""
        session_id = str(uuid.uuid4())
        self.sessions[session_id] = {
            'user_id': user_id,
            'mode': mode,
            'created_at': datetime.now(),
            'message_count': 0
        }
        self.user_connections[user_id].add(session_id)
        return session_id
    
    def cleanup_session(self, session_id: str):
        """Clean up session resources."""
        if session_id in self.sessions:
            user_id = self.sessions[session_id]['user_id']
            self.user_connections[user_id].discard(session_id)
            del self.sessions[session_id]
    
    def check_rate_limit(self, user_id: str) -> tuple:
        """Check if user is within rate limits."""
        # Connection limit
        if len(self.user_connections[user_id]) >= self.MAX_CONNECTIONS_PER_USER:
            return False, "Max connections exceeded"
        
        # Request rate limit
        now = datetime.now()
        recent = [ts for ts in self.user_requests[user_id] 
                  if (now - ts).total_seconds() < 60]
        self.user_requests[user_id] = recent
        
        if len(recent) >= self.MAX_REQUESTS_PER_MINUTE:
            return False, "Rate limit exceeded"
        
        self.user_requests[user_id].append(now)
        return True, "OK"
    
    def get_stats(self) -> dict:
        """Get session statistics."""
        return {
            'total_sessions': len(self.sessions),
            'unique_users': len(self.user_connections),
            'sessions_by_mode': {
                mode: sum(1 for s in self.sessions.values() if s['mode'] == mode)
                for mode in ['voice', 'text']
            }
        }

# Demo
manager = SessionManager()

print("Session Management Demo:")
print("="*50)

# Create sessions
session1 = manager.create_session('user-alice', 'voice')
session2 = manager.create_session('user-alice', 'text')
session3 = manager.create_session('user-bob', 'voice')

print(f"Created session for Alice (voice): {session1[:8]}...")
print(f"Created session for Alice (text): {session2[:8]}...")
print(f"Created session for Bob (voice): {session3[:8]}...")

print(f"\nStatistics: {manager.get_stats()}")

# Rate limit check
allowed, msg = manager.check_rate_limit('user-alice')
print(f"\nRate limit check for Alice: {allowed} - {msg}")

# Cleanup
manager.cleanup_session(session1)
print(f"\nAfter cleanup: {manager.get_stats()}")

---
## 6. Message Protocol

### Azure Realtime API Message Types

The Realtime API uses JSON messages for control and base64-encoded audio.

#### Client → Azure Messages

| Message Type | Purpose |
|--------------|--------|
| `session.update` | Configure session (voice, instructions) |
| `input_audio_buffer.append` | Send audio chunks |
| `input_audio_buffer.commit` | Commit audio for processing |
| `response.create` | Request a response |

#### Azure → Client Messages

| Message Type | Purpose |
|--------------|--------|
| `session.created` | Session initialized |
| `session.updated` | Session config confirmed |
| `input_audio_buffer.speech_started` | Voice activity detected |
| `input_audio_buffer.speech_stopped` | Voice activity ended |
| `response.audio.delta` | Audio chunk of response |
| `response.audio_transcript.delta` | Transcript of response |
| `response.done` | Response complete |
| `error` | Error occurred |

In [ ]:
import json

# Message Protocol Examples

# 1. Session Configuration
session_update = {
    "type": "session.update",
    "session": {
        "modalities": ["text", "audio"],
        "instructions": "You are a helpful assistant. Respond naturally and concisely.",
        "voice": "alloy",  # Options: alloy, echo, shimmer
        "input_audio_format": "pcm16",
        "output_audio_format": "pcm16",
        "input_audio_transcription": {
            "model": "whisper-1"
        },
        "turn_detection": {
            "type": "server_vad",  # Server-side voice activity detection
            "threshold": 0.5,
            "prefix_padding_ms": 300,
            "silence_duration_ms": 200
        }
    }
}

# 2. Send Audio Buffer
audio_append = {
    "type": "input_audio_buffer.append",
    "audio": "BASE64_ENCODED_PCM16_AUDIO_DATA"
}

# 3. Response from Azure (example)
azure_response = {
    "type": "response.audio.delta",
    "response_id": "resp_ABC123",
    "item_id": "item_XYZ",
    "output_index": 0,
    "content_index": 0,
    "delta": "BASE64_ENCODED_AUDIO_RESPONSE"
}

# 4. Error Message
error_message = {
    "type": "error",
    "error": {
        "type": "invalid_request_error",
        "code": "invalid_audio_format",
        "message": "Audio must be pcm16 format",
        "param": "audio"
    }
}

print("Message Protocol Examples:")
print("="*50)

print("\n1. Session Update (Client → Azure):")
print(json.dumps(session_update, indent=2)[:500] + "...")

print("\n2. Audio Append (Client → Azure):")
print(json.dumps(audio_append, indent=2))

print("\n3. Audio Response (Azure → Client):")
print(json.dumps(azure_response, indent=2))

---
## 7. Security Best Practices

### 🔐 Key Security Principles

1. **Never expose API keys to the client**
   - All Azure calls go through the backend proxy
   - Keys stored in environment variables on server

2. **Implement authentication**
   - Validate user tokens before allowing WebSocket connections
   - Use JWT or OAuth2 in production

3. **Rate limiting**
   - Limit connections per user
   - Limit requests per time window
   - Prevent abuse and control costs

4. **Input validation**
   - Validate message formats
   - Sanitize any user content
   - Limit message sizes

5. **Use secure connections**
   - WSS (WebSocket Secure) only
   - HTTPS for any HTTP endpoints

In [ ]:
# Security Implementation Patterns

from datetime import datetime, timedelta
from typing import Optional
import hashlib
import hmac
import secrets

class SecurityManager:
    """
    Security utilities for voice chat application.
    """
    
    def __init__(self, secret_key: str):
        self.secret_key = secret_key.encode()
        self.token_expiry = timedelta(hours=1)
    
    def generate_session_token(self, user_id: str) -> str:
        """
        Generate a secure session token.
        In production, use JWT with proper claims.
        """
        timestamp = datetime.utcnow().isoformat()
        data = f"{user_id}:{timestamp}"
        signature = hmac.new(
            self.secret_key, 
            data.encode(), 
            hashlib.sha256
        ).hexdigest()
        return f"{data}:{signature}"
    
    def validate_token(self, token: str) -> Optional[str]:
        """
        Validate a session token and return user_id if valid.
        """
        try:
            parts = token.rsplit(':', 1)
            if len(parts) != 2:
                return None
            
            data, signature = parts
            expected_sig = hmac.new(
                self.secret_key, 
                data.encode(), 
                hashlib.sha256
            ).hexdigest()
            
            if not hmac.compare_digest(signature, expected_sig):
                return None
            
            user_id, timestamp = data.rsplit(':', 1)
            # Check expiry (simplified)
            return user_id
        except Exception:
            return None
    
    @staticmethod
    def validate_audio_message(message: dict) -> tuple:
        """
        Validate an audio message structure.
        """
        required_fields = ['type']
        
        for field in required_fields:
            if field not in message:
                return False, f"Missing required field: {field}"
        
        # Validate audio size if present
        if 'audio' in message:
            max_audio_size = 1024 * 1024  # 1MB
            if len(message['audio']) > max_audio_size:
                return False, "Audio data exceeds maximum size"
        
        return True, "Valid"

# Demo
security = SecurityManager("my-super-secret-key-change-in-production")

print("Security Demo:")
print("="*50)

# Generate token
token = security.generate_session_token("user-alice")
print(f"Generated token: {token[:50]}...")

# Validate token
user_id = security.validate_token(token)
print(f"Validated user: {user_id}")

# Validate message
test_msg = {"type": "input_audio_buffer.append", "audio": "SGVsbG8="}
valid, msg = SecurityManager.validate_audio_message(test_msg)
print(f"Message validation: {valid} - {msg}")

---
## 8. Code Examples

### Complete WebSocket Proxy Pattern

The core pattern for proxying WebSocket connections between a client and Azure is **bidirectional forwarding**:

```python
async def handle_voice_session(client_ws, azure_url):
    async with websockets.connect(azure_url) as azure_ws:
        # Run both directions concurrently
        await asyncio.gather(
            proxy_client_to_azure(client_ws, azure_ws),
            proxy_azure_to_client(azure_ws, client_ws)
        )
```

This ensures:
- Audio from client reaches Azure in real-time
- Azure's responses reach client with minimal latency
- Both connections are properly managed

In [ ]:
# Complete WebSocket Server Example (Conceptual)
# This shows the full structure but won't run without websockets library

WEBSOCKET_SERVER_CODE = '''
import asyncio
import websockets
import json
import os

# Configuration from environment
AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT = os.getenv('AZURE_REALTIME_DEPLOYMENT')
API_VERSION = '2024-10-01-preview'

def build_azure_url():
    """Build Azure Realtime API WebSocket URL."""
    ws_endpoint = AZURE_ENDPOINT.replace('https://', 'wss://').rstrip('/')
    return (
        f"{ws_endpoint}/openai/realtime"
        f"?api-version={API_VERSION}"
        f"&deployment={AZURE_DEPLOYMENT}"
        f"&api-key={AZURE_API_KEY}"
    )

async def proxy_client_to_azure(client_ws, azure_ws):
    """Forward messages from browser to Azure."""
    async for message in client_ws:
        await azure_ws.send(message)

async def proxy_azure_to_client(azure_ws, client_ws):
    """Forward messages from Azure to browser."""
    async for message in azure_ws:
        await client_ws.send(message)

async def handle_connection(client_ws):
    """Handle a new WebSocket connection."""
    print(f"New client connected")
    
    azure_url = build_azure_url()
    
    async with websockets.connect(
        azure_url,
        max_size=10 * 1024 * 1024,  # 10MB
        ping_interval=20
    ) as azure_ws:
        print(f"Connected to Azure Realtime API")
        
        # Bidirectional proxy
        await asyncio.gather(
            proxy_client_to_azure(client_ws, azure_ws),
            proxy_azure_to_client(azure_ws, client_ws),
            return_exceptions=True
        )

async def main():
    """Start the WebSocket server."""
    async with websockets.serve(
        handle_connection,
        "0.0.0.0",
        8001,
        max_size=10 * 1024 * 1024
    ):
        print("Voice Chat Server running on ws://0.0.0.0:8001")
        await asyncio.Future()  # Run forever

if __name__ == "__main__":
    asyncio.run(main())
'''

print("Complete WebSocket Server Structure:")
print("="*50)
print(WEBSOCKET_SERVER_CODE)

In [ ]:
# Text Mode: Using Microsoft Agent Framework
# The voice chat app uses Agent Framework for text mode (same patterns as .NET)

import json

TEXT_CHAT_CODE = '''
# Text mode with Microsoft Agent Framework
# This replaces direct REST API calls with the unified Agent pattern
# Pattern from: https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai

from agent_framework.azure import AzureOpenAIChatClient
import os

async def setup_text_mode():
    """Initialize ChatAgent for text mode chat."""
    
    # Create Azure OpenAI chat client with explicit settings
    # (like .NET's AzureOpenAIClient.GetChatClient)
    client = AzureOpenAIChatClient(
        endpoint=os.getenv('AZURE_ENDPOINT'),
        deployment_name=os.getenv('AZURE_CHAT_DEPLOYMENT', 'gpt-4o'),
        api_key=os.getenv('AZURE_API_KEY'),
    )
    
    # Create the agent (like .NET's CreateAIAgent)
    agent = client.create_agent(
        instructions="You are a helpful assistant.",
    )
    
    return agent


async def handle_text_session(websocket, agent):
    """Handle a text chat session with conversation memory."""
    
    # Create session for this session (maintains context across turns)
    session = await agent.create_session()
    
    async for message in websocket:
        data = json.loads(message)
        user_message = data.get('content', '')
        
        # Run agent and get response (with conversation history via thread)
        result = await agent.run(user_message, session=session)
        
        await websocket.send(json.dumps({
            'type': 'text_response',
            'content': str(result)
        }))


# For streaming responses (better UX for long responses):
async def handle_text_session_streaming(websocket, agent):
    """Handle text chat with streaming responses."""
    
    session = await agent.create_session()
    
    async for message in websocket:
        data = json.loads(message)
        user_message = data.get('content', '')
        
        # Stream response chunks
        response_text = ""
        async for chunk in agent.run_stream(user_message, session=session):
            if chunk.text:
                response_text += chunk.text
                # Optionally send partial updates to client
        
        await websocket.send(json.dumps({
            'type': 'text_response',
            'content': response_text
        }))


# Benefits over direct API calls:
# 1. Automatic conversation history management via AgentSession
# 2. Built-in streaming support with run_stream()
# 3. Easy tool/function integration
# 4. Same patterns as .NET implementation
# 5. Environment variable auto-loading from AZURE_OPENAI_* vars
'''

print("Text Chat with Microsoft Agent Framework:")
print("="*50)
print(TEXT_CHAT_CODE)

---
## 📝 Summary

### Key Concepts Learned

1. **WebSocket Architecture**: Bidirectional, persistent connections for real-time communication

2. **Proxy Pattern**: Backend server acts as intermediary between browser and Azure for security

3. **Audio Processing**: PCM16 at 24kHz, base64 encoded for JSON transport

4. **Session Management**: Track users, connections, and implement rate limiting

5. **Message Protocol**: JSON-based control messages with specific types for each action

6. **Security**: Never expose API keys, validate inputs, use secure connections

7. **Microsoft Agent Framework**: `AzureOpenAIChatClient.create_agent()` for text mode with conversation memory

### Agent Framework Packages

```bash
# The --pre flag is required while Agent Framework is in preview
pip install agent-framework-core agent-framework-azure-ai --pre
```

### Key Imports

```python
from agent_framework.azure import AzureOpenAIChatClient

# Create client and agent
client = AzureOpenAIChatClient(
    endpoint="https://your-resource.openai.azure.com",
    deployment_name="gpt-4o",
    api_key="your-api-key",
)

agent = client.create_agent(
    instructions="You are a helpful assistant.",
)

# Use session for multi-turn conversation
session = await agent.create_session()
result = await agent.run("Hello!", session=session)
```

### Cross-Platform Consistency

The same Agent Framework patterns work in both Python and .NET:

| Python | .NET |
|--------|------|
| `AzureOpenAIChatClient` | `AzureOpenAIClient` |
| `client.create_agent()` | `chatClient.CreateAIAgent()` |
| `agent.run()` | `agent.RunAsync()` |
| `agent.run_stream()` | `agent.RunStreamingAsync()` |
| `agent.create_session()` | `agent.CreateSessionAsync()` |

### Next Steps

- Run the actual voice chat application to see these concepts in action
- Explore the browser-side code for audio capture and playback
- Experiment with different voice settings and system prompts
- Try adding tools/functions to the agent for enhanced capabilities

### Resources

- [Azure OpenAI Realtime API Documentation](https://learn.microsoft.com/azure/ai-services/openai/realtime-audio-quickstart)
- [Microsoft Agent Framework - Python Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai)
- [Microsoft Agent Framework - GitHub](https://github.com/microsoft/agent-framework)
- [WebSocket Protocol RFC](https://datatracker.ietf.org/doc/html/rfc6455)
- [Python websockets library](https://websockets.readthedocs.io/)